## TYTANチュートリアル（温度計）

2023年5月5日

ビネクラ安田

出典：[量子アニーリング（D-wave）で温度計パズル（Thermometers）を解く](https://vigne-cla.com/21-16/)

### 問題
温度計パズル（Thermometers）と呼ばれるパズル。

各行、各列とも数字の数だけ赤マスとして塗るが、温度計なので球部（水銀溜まり）から順にしか塗られないという制限がある。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-16_1-300x300.png" width = 20%>
</div>

答えはこちら。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-16_2-300x300.png" width = 20%>
</div>

### QUBOモデルでは何が設定できるか？（おさらい）

**<font color="red">「n個の量子ビットからm個を1にする」</font>**

例）3個の量子ビットから2個を1にする
```
H = (q1 + q2 + q3 - 2)**2
```

**<font color="red">「2個の量子ビットを降順にする」</font>**

例）q0 ≧ q1 にする（＝[0, 1] になったらペナルティを与える）
```
H = (1 - q0) * q1
```

**<font color="red">「すべての量子ビットを降順にする」</font>**

例）q0 ≧ q1 ≧ q2 にする
```
H = (1 - q0) * q1
H += (1 - q1) * q2

```

初めて降順の設定を使います。その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

### 制約条件

16個の量子ビットを各マスに対応させる。結果が１なら色を塗ることとする。

まず、各行、各列とも指定の個数だけ１になるように設定。基本の「ｎ個の量子ビットからｍ個を１にする」を使う。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-16_3r.png" width = 45%>
</div>

次に、各温度計に着目して、球部から降順になるように設定。ここで言う降順とは q0 ≧ q1 ≧ q2 の意味で、[1, 1, 0] のように途中から0に切り替わる形だけでなく、[0, 0, 0]、[1, 1, 1] のようにずっと等しくてもOK。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-16_4.png" width = 45%>
</div>

## コード１

In [ ]:
!pip install git+https://github.com/tytansdk/tytan
!pip install pyqubo

In [5]:
from tytan import *
import numpy as np
from pyqubo import Binary

#量子ビットを用意する
q00 = Binary('q00')
q01 = Binary('q01')
q02 = Binary('q02')
q03 = Binary('q03')
q04 = Binary('q04')
q05 = Binary('q05')
q06 = Binary('q06')
q07 = Binary('q07')
q08 = Binary('q08')
q09 = Binary('q09')
q10 = Binary('q10')
q11 = Binary('q11')
q12 = Binary('q12')
q13 = Binary('q13')
q14 = Binary('q14')
q15 = Binary('q15')

#各行、「4個から指定の個数だけ1になる」
H = 0
H += (q00 + q01 + q02 + q03 - 2)**2
H += (q04 + q05 + q06 + q07 - 1)**2
H += (q08 + q09 + q10 + q11 - 3)**2
H += (q12 + q13 + q14 + q15 - 1)**2

#各列、「4個から指定の個数だけ1になる」
H += (q00 + q04 + q08 + q12 - 3)**2
H += (q01 + q05 + q09 + q13 - 1)**2
H += (q02 + q06 + q10 + q14 - 1)**2
H += (q03 + q07 + q11 + q15 - 2)**2

#各温度計、球部から降順になる
H += (1 - q08) * q04
H += (1 - q04) * q00 #8→4→0の連鎖

H += (1 - q05) * q01

H += (1 - q03) * q02

H += (1 - q07) * q06

H += (1 - q11) * q10
H += (1 - q10) * q09 #11→10→9の連鎖

H += (1 - q13) * q12

H += (1 - q15) * q14


#コンパイル
model = H.compile()
qubo, offset = model.to_qubo()

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo, shots=500)

#上位5件
for r in result[:5]:
    print(r)

#上位2件
print(np.array(list(result[0][0].values()), int).reshape(4, 4))
print(np.array(list(result[1][0].values()), int).reshape(4, 4))

[{'q00': 1.0, 'q01': 0.0, 'q02': 0.0, 'q03': 1.0, 'q04': 1.0, 'q05': 0.0, 'q06': 0.0, 'q07': 0.0, 'q08': 1.0, 'q09': 0.0, 'q10': 1.0, 'q11': 1.0, 'q12': 0.0, 'q13': 1.0, 'q14': 0.0, 'q15': 0.0}, -30.0, 194]
[{'q00': 1.0, 'q01': 0.0, 'q02': 0.0, 'q03': 1.0, 'q04': 1.0, 'q05': 0.0, 'q06': 0.0, 'q07': 0.0, 'q08': 1.0, 'q09': 1.0, 'q10': 1.0, 'q11': 0.0, 'q12': 0.0, 'q13': 0.0, 'q14': 0.0, 'q15': 1.0}, -29.0, 59]
[{'q00': 1.0, 'q01': 1.0, 'q02': 0.0, 'q03': 0.0, 'q04': 1.0, 'q05': 0.0, 'q06': 0.0, 'q07': 0.0, 'q08': 1.0, 'q09': 0.0, 'q10': 1.0, 'q11': 1.0, 'q12': 0.0, 'q13': 0.0, 'q14': 0.0, 'q15': 1.0}, -29.0, 119]
[{'q00': 0.0, 'q01': 0.0, 'q02': 1.0, 'q03': 1.0, 'q04': 1.0, 'q05': 0.0, 'q06': 0.0, 'q07': 0.0, 'q08': 1.0, 'q09': 0.0, 'q10': 0.0, 'q11': 1.0, 'q12': 0.0, 'q13': 1.0, 'q14': 0.0, 'q15': 0.0}, -28.0, 7]
[{'q00': 0.0, 'q01': 0.0, 'q02': 1.0, 'q03': 1.0, 'q04': 1.0, 'q05': 0.0, 'q06': 0.0, 'q07': 0.0, 'q08': 1.0, 'q09': 0.0, 'q10': 0.0, 'q11': 1.0, 'q12': 1.0, 'q13': 1.0, 'q14'

一番目の解は模範解答の通りである。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-16_2-300x300.png" width = 20%>
</div>